[구글 코랩(Colab)에서 실행하기](https://colab.research.google.com/github/lovedlim/bigdata_analyst_cert/blob/main/part4/ch6/p6_type2.ipynb)

# 1. 문제정의
- 도서 참고

# 2. 라이브러리 및 데이터 불러오기

In [24]:
# 데이터 불러오기
import pandas as pd
# train = pd.read_csv("energy_train.csv")
# test = pd.read_csv("energy_test.csv")
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch6/energy_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch6/energy_test.csv")

# 3. 탐색적 데이터 분석(EDA)

In [25]:
# 데이터 크기 확인
train.shape, test.shape

((537, 10), (231, 9))

In [26]:
# train 샘플 확인
train.head()

,Compac,Surf_Area,Wall_Area,Roof,Height,Orient,Glaze_Area,Glaze_Distr,Cool_Load,Heat_Load
0,0.74,686.0,245.0,220.5,Short,South,0.25,3,14.72,Very Low
1,0.98,514.5,294.0,Small,Tall,South,0.40,2,33.94,High
2,0.90,563.5,318.5,Medium,Tall,West,0.25,1,37.58,High
3,0.74,686.0,245.0,220.5,Short,South,0.25,2,15.18,Very Low
4,0.98,514.5,294.0,Small,Tall,East,0.25,2,29.69,High


In [27]:
# test 샘플 확인
test.head()

,Compac,Surf_Area,Wall_Area,Roof,Height,Orient,Glaze_Area,Glaze_Distr,Cool_Load
0,0.64,784.0,343.0,220.5,Short,South,0.40,4,22.25
1,0.82,612.5,318.5,Large,Tall,North,0.40,3,32.43
2,0.76,661.5,416.5,Medium,Tall,South,0.10,5,33.64
3,0.74,686.0,245.0,220.5,Short,East,0.40,1,17.25
4,0.64,784.0,343.0,220.5,Short,North,0.25,5,20.13


In [28]:
# type 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537 entries, 0 to 536
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Compac       537 non-null    float64
 1   Surf_Area    537 non-null    float64
 2   Wall_Area    537 non-null    float64
 3   Roof         537 non-null    object 
 4   Height       537 non-null    object 
 5   Orient       537 non-null    object 
 6   Glaze_Area   537 non-null    float64
 7   Glaze_Distr  537 non-null    int64  
 8   Cool_Load    537 non-null    float64
 9   Heat_Load    537 non-null    object 
dtypes: float64(5), int64(1), object(4)
memory usage: 42.1+ KB


In [29]:
train.describe(include='object')

,Roof,Height,Orient,Heat_Load
count,537,537,537,537
unique,4,2,4,5
top,220.5,Tall,South,Very Low
freq,257,280,145,142


In [30]:
test.describe(include='object')

,Roof,Height,Orient
count,231,231,231
unique,4,2,4
top,220.5,Short,North
freq,127,127,74


In [31]:
# 결측치 확인(train)
train.isnull().sum().sum()

0

In [32]:
# 결측치 확인(test)
test.isnull().sum().sum()

0

In [33]:
# target 확인
train['Heat_Load'].value_counts()

,count
Heat_Load,
Very Low,142
Low,123
High,122
Very High,79
Medium,71


# 4. 데이터 전처리

In [34]:
# target컬럼이 object형이라 제외
target = train.pop('Heat_Load')

In [35]:
# 원핫 인코딩(판다스)
print(train.shape, test.shape)
train = pd.get_dummies(train)
test = pd.get_dummies(test)
print(train.shape, test.shape)

(537, 9) (231, 9)
(537, 16) (231, 16)


# 5. 검증 데이터 분할

In [36]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train,
                                            target,
                                            test_size=0.2,
                                            random_state=0)
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((429, 16), (108, 16), (429,), (108,))

# 6. 머신러닝 학습 및 평가

In [37]:
# 라이브러리 불러오기
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import f1_score

In [38]:
# 의사결정나무
dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_tr, y_tr)
pred = dt.predict(X_val)
print(f1_score(y_val, pred, average='macro'))

0.9167995817564094


In [39]:
# 랜덤포레스트
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
print(f1_score(y_val, pred, average='macro'))

0.9277616846430405


In [40]:
# Xgboost
# xg = xgb.XGBClassifier(random_state=0)
# xg.fit(X_tr, y_tr)
# pred = xg.predict(X_val)
# print(f1_score(y_val, pred, average='macro'))

# [에러] ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4], got ['High' 'Low' 'Medium' 'Very High' 'Very Low']

In [41]:
# target 인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_tr_adjusted = le.fit_transform(y_tr)

# 모델 학습 및 예측
xg = xgb.XGBClassifier(random_state=0)
xg.fit(X_tr, y_tr_adjusted)
pred = xg.predict(X_val)

# 예측값을 원래 문자로 변경
pred = le.inverse_transform(pred)

print(f1_score(y_val, pred, average='macro'))

0.9374839068652628


In [42]:
# LightGBM
lg = lgb.LGBMClassifier(random_state=0, verbose=-1)
lg.fit(X_tr, y_tr)
pred = lg.predict(X_val)
print(f1_score(y_val, pred, average='macro'))

0.9319703995747777


# 7. 예측 및 결과 파일 생성

In [43]:
# test 예측
pred = xg.predict(test)
submit = pd.DataFrame({
    'pred': pred
})
submit.to_csv("result.csv", index=False)

In [44]:
pd.read_csv("result.csv")

,pred
0,1
1,0
2,0
3,1
4,1
...,...
226,4
227,1
228,4
229,1


# 8. 심화 학습

In [45]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# train = pd.read_csv("energy_train.csv")
# test = pd.read_csv("energy_test.csv")
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch6/energy_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch6/energy_test.csv")

# 데이터 전처리
target = train.pop('Heat_Load')
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# 모델 준비
rf = RandomForestClassifier(random_state=0)

# 크로스 밸리데이션 수행(5-fold)
scores = cross_val_score(rf, train, target, cv=5, scoring='f1_macro')
print(scores)

# 평균 F1 스코어 출력
print(scores.mean())

[0.91726901 0.90856297 0.92346084 0.95283204 0.95716959]
0.9318588885917534


In [46]:
# 최종 제출 파일
rf.fit(train, target)
pred = rf.predict(test)
submit = pd.DataFrame({'pred': pred})
submit.to_csv("result.csv", index=False)